### Don't feel like cleaning out the other Notebook, this will import a db from my PC and from my Laptop, hoping that a model could see the difference and sort of treat it like a different user/recognize it

In [ ]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

dfPC = pd.read_sql("SELECT * FROM TransformedData", sqlite3.connect("../data/transformed_outputPC.db"))
dfLaptop = pd.read_sql("SELECT * FROM TransformedData", sqlite3.connect("../data/transformed_outputLaptop.db"))



df_total = pd.concat([dfPC, dfLaptop])
#dfPctest = df_total.iloc[[33]]
#df_total = df_total.drop(df_total.index[33])
#dfLaptoptest = df_total.iloc[[84]]
#df_total = df_total.drop(df_total.index[84])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X = df_total.drop(columns=["user_id", "message_id"])
y = le.fit_transform(df_total['user_id'])  # y will now be numeric

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Use stratified k-fold to maintain class balance
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define F1 scorer
f1 = make_scorer(f1_score, average='macro')

# Cross-validate and get F1 scores
f1_scores = cross_val_score(model, X, y, scoring=f1, cv=cv)
model.fit(X_train, y_train)

print("F1 scores per fold:", f1_scores)
print("Mean F1 score:", f1_scores.mean())

### Incase you forget 
Precision = How many predictions of Class X where actually class X
Recall = How many instances of class X where correctly predicted (false negatives included)
F1-Score = balance between the first 2, balancing false negatives and false positives
Support = How many of Class X were in the dataset

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

### Support of 10/11 is very small still, but it shows some kind of promise. The Model is not just randomly doing things, it actually has a solid result at this stage, You can't see this but its recall for the 2 classes that currently exist are 0.73 and 0.82